In [1]:
import sys # for automation and parallelization: set manual to false when run by a launcher
import json
default = {'scenario': 'base', 'training_folder':'../..'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [3]:
scenario = argv['scenario']


on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)

training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if not on_lambda:
    scenario_folder = training_folder + '/scenarios/' + scenario + '/inputs/'
    output_folder = training_folder + '/scenarios/' + scenario  + '/outputs/'
    model_folder = training_folder + '/scenarios/' + scenario  + '/model/'
else:
    scenario_folder = input_folder
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/base/outputs/
scen folder :  ../../scenarios/base/inputs/
model folder :  ../../scenarios/base/model/


In [4]:
if 'params' in argv.keys():
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())
else:
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)

# inputs

In [5]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')

zones: 100%|██████████| 25/25 [00:00<00:00, 121.02it/s]    


# pathfinder

In [6]:
sm.road_links['time_ff'] = sm.road_links['length']/(sm.road_links['speed']*1000/3600)
sm.step_road_pathfinder(method='aon', access_time='time', time_column= 'time_ff',path_analysis=False)

self.volumes does not exist. od generated with self.zones


In [7]:
od_set = set()
for o in set(sm.zone_to_transit["a"]).intersection(sm.zones.index):
    for d in set(sm.zone_to_transit["a"]).intersection(sm.zones.index):
        od_set.add((o,d))

In [8]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    cutoff = var['pathfinder']['cutoff'],   
    path_analysis=False,
    od_set= od_set,
    #engine='b', # b is faster!
)


start publicpathfinder
58 sources 58 targets direct search
path_analysis


In [9]:
sm.car_los.drop('path', axis=1, inplace=True)
sm.pt_los.drop('path', axis=1, inplace=True)

In [10]:
sm.pt_los = sm.pt_los.rename(columns ={"gtime":"time"})

In [11]:
sm.to_zippedpickles(model_folder +'aon_los', only_attributes=['car_los','pt_los','zones','zones_before_clustering', 'zones_poly'])

pt_los: 100%|██████████| 28/28 [00:00<00:00, 1629.24it/s]


In [12]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined